# TruLens-Canopy Quickstart

 Canopy is an open-source framework and context engine built on top of the Pinecone vector database so you can build and host your own production-ready chat assistant at any scale. By integrating TruLens into your Canopy assistant, you can quickly iterate on and gain confidence in the quality of your chat assistant.

In [521]:
# !pip install -qU canopy-sdk trulens-eval cohere ipywidgets

In [522]:
import numpy
assert numpy.__version__ >= "1.26", "Numpy version did not updated, if you are working on Colab please restart the session."

## Set Keys

In [523]:
import os

os.environ["PINECONE_API_KEY"] = "YOUR_PINECONE_API_KEY"  # take free trial key from https://app.pinecone.io/
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # take free trial key from https://platform.openai.com/api-keys
os.environ["CO_API_KEY"] = "YOUR_COHERE_API_KEY"  # take free trial key from https://dashboard.cohere.com/api-keys

In [525]:
assert os.environ["PINECONE_API_KEY"] != "YOUR_PINECONE_API_KEY", "please provide PINECONE API key"
assert os.environ["OPENAI_API_KEY"] != "YOUR_OPENAI_API_KEY", "please provide OpenAI API key"
assert os.environ["CO_API_KEY"] != "YOUR_COHERE_API_KEY", "please provide Cohere API key"

## Load data
Downloading Pinecone's documentation as data to ingest to our Canopy chatbot:

In [526]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


In [527]:
print(data["text"][50][:847].replace("\n\n", "\n").replace("[Suggest Edits](/edit/limits)", "") + "\n......")
print("source: ", data["source"][50])

# Limits
This is a summary of current Pinecone limitations. For many of these, there is a workaround or we're working on increasing the limits.

## Upserts

Max vector dimensionality is 20,000.

Max size for an upsert request is 2MB. Recommended upsert limit is 100 vectors per request.

Vectors may not be visible to queries immediately after upserting. You can check if the vectors were indexed by looking at the total with `describe_index_stats()`, although this method may not work if the index has multiple replicas. Pinecone is eventually consistent.

Pinecone supports sparse vector values of sizes up to 1000 non-zero values.

## Queries

Max value for `top_k`, the number of results to return, is 10,000. Max value for `top_k` for queries with `include_metadata=True` or `include_data=True` is 1,000.

......
source:  https://docs.pinecone.io/docs/limits


## Setup Tokenizer

In [528]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

['Hello', ' world', '!']

## Create and Load Index

In [529]:
from tqdm.auto import tqdm
from canopy.knowledge_base import KnowledgeBase
from canopy.models.data_models import Document
from canopy.knowledge_base import list_canopy_indexes

index_name = "pinecone-docs"

kb = KnowledgeBase(index_name)

if not any(name.endswith(index_name) for name in list_canopy_indexes()):
    kb.create_canopy_index()

kb.connect()

documents = [Document(**row) for _, row in data.iterrows()]

batch_size = 100

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/1 [00:00<?, ?it/s]

## Create context and chat engine

In [530]:
from canopy.models.data_models import Query
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

## Instrument static methods used by engine with TruLens 

In [531]:
warnings.filterwarnings('ignore')
from trulens_eval.tru_custom_app import instrument

from canopy.context_engine import ContextEngine
instrument.method(ContextEngine, "query")

from canopy.chat_engine import ChatEngine
instrument.method(ChatEngine, "chat")

## Create feedback functions using instrumented methods

In [532]:
from trulens_eval import Tru
tru = Tru(database_redact_keys=True)

In [533]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI
import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

intput = Select.RecordCalls.chat.args.messages[0].content
context = Select.RecordCalls.context_engine.query.rets.content.root[:].snippets[:].text
output = Select.RecordCalls.chat.rets.choices[0].message.content

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness", higher_is_better=True)
    .on(context.collect())
    .on(output)
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance", higher_is_better=True)
    .on(intput)
    .on(output)
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance", higher_is_better=True)
    .on(intput)
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.context_engine.query.rets.content.root[:].snippets[:].text.collect() .
✅ In Groundedness, input statement will be set to __record__.app.chat.rets.choices[0].message.content .
✅ In Answer Relevance, input prompt will be set to __record__.app.chat.args.messages[0].content .
✅ In Answer Relevance, input response will be set to __record__.app.chat.rets.choices[0].message.content .
✅ In Context Relevance, input question will be set to __record__.app.chat.args.messages[0].content .
✅ In Context Relevance, input statement will be set to __record__.app.context_engine.query.rets.content.root[:].snippets[:].text .


## Create recorded app and run it

In [534]:
from trulens_eval import TruCustomApp

app_id = "canopy default"
tru_recorder = TruCustomApp(chat_engine, app_id=app_id, feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function ContextEngine.query at 0x13a715630> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14b157ac0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function ChatEngine.chat at 0x13a587eb0> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14b157ac0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function QueryGenerator.generate at 0x13a7157e0> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14b157ac0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [535]:
from canopy.models.data_models import UserMessage

queries = [
    [UserMessage(content="What is the maximum dimension for a dense vector in Pinecone?")],
    [UserMessage(content="How can you get started with Pinecone and TruLens?")],
    [UserMessage(content="What is the the maximum top-k for a query to Pinecone?")]
]

answers = []

for query in queries:
    with tru_recorder as recording:
        response = chat_engine.chat(query)
        answers.append(response.choices[0].message.content)

Unsure what the main input string is for the call to chat with args [[UserMessage(role=<Role.USER: 'user'>, content='What is the maximum dimension for a dense vector in Pinecone?')]].
Unsure what the main output string is for the call to chat with return type <class 'canopy.models.api_models.ChatResponse'>.
Unsure what the main input string is for the call to chat with args [[UserMessage(role=<Role.USER: 'user'>, content='How can you get started with Pinecone and TruLens?')]].
Unsure what the main output string is for the call to chat with return type <class 'canopy.models.api_models.ChatResponse'>.
Unsure what the main input string is for the call to chat with args [[UserMessage(role=<Role.USER: 'user'>, content='What is the the maximum top-k for a query to Pinecone?')]].
Unsure what the main output string is for the call to chat with return type <class 'canopy.models.api_models.ChatResponse'>.


As you can see, we got the wrong answer, the limits for sparse vectors instead of dense vectors:

In [536]:
print(queries[0][0].content + "\n")
print(answers[0])

What is the maximum dimension for a dense vector in Pinecone?

The maximum dimension for a dense vector in Pinecone is over 4 billion dimensions.


In [556]:
tru.get_leaderboard(app_ids=[app_id])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
canopy default,0.806667,0.5,0.966667,4.0,0.002296


## Run Canopy with Cohere reranker

In [538]:
from canopy.knowledge_base.reranker.cohere import CohereReranker

kb = KnowledgeBase(index_name=index_name, reranker=CohereReranker(top_n=3), default_top_k=30)
kb.connect()

reranker_chat_engine = ChatEngine(ContextEngine(kb))


In [539]:
reranking_app_id = "canopy_reranking"
reranking_tru_recorder = TruCustomApp(reranker_chat_engine,
                                      app_id=reranking_app_id,
                                      feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

answers = []

for query in queries:
    with reranking_tru_recorder as recording:
        answers.append(reranker_chat_engine.chat(query).choices[0].message.content)

Function <function ContextEngine.query at 0x13a715630> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14c878ca0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function ChatEngine.chat at 0x13a587eb0> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14c878ca0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function QueryGenerator.generate at 0x13a7157e0> was not found during instrumentation walk. Make sure it is accessible by traversing app <canopy.chat_engine.chat_engine.ChatEngine object at 0x14c878ca0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Unsure what the main input string is for the call to chat with args [[UserMessage(role=<

With reranking we get the right answer!

In [540]:
print(queries[0][0].content + "\n")
print(answers[0])

What is the maximum dimension for a dense vector in Pinecone?

The maximum dimension for a dense vector in Pinecone is 20,000. (Source: https://docs.pinecone.io/docs/limits)


## Evaluate the effect of reranking 

In [569]:
tru.get_leaderboard(app_ids=[app_id, reranking_app_id])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
canopy_reranking,0.830556,0.833333,0.966667,4.0,0.002265
canopy default,0.806667,0.500000,0.966667,4.0,0.002296
